## Tkinter Game

Tkinter is the standard GUI library for Python. Python when combined with Tkinter provides a fast and easy way to create GUI applications. Tkinter provides a powerful object-oriented interface to the Tk GUI toolkit.

## Tkinter - BrickBreaker 彈跳球

In [1]:
import tkinter as tk

class PlayComponent(object):
    def __init__(self, canvas, item):
        self.item = item
        self.canvas = canvas

    def move(self, x, y):
        self.canvas.move(self.item, x, y)

    def position(self):
        return self.canvas.coords(self.item)

    def delete(self):
        self.canvas.delete(self.item)

class Paddle(PlayComponent):
    def __init__(self, canvas, x, y):
        self.height = 5
        self.width = 100
        self.ball = None
        item = canvas.create_rectangle(x - self.width/2, y - self.height/2,
                                       x + self.width/2, y + self.height/2, fill='green')
        super(Paddle, self).__init__(canvas, item)

    def set_ball(self, ball):
        self.ball = ball

    def move(self, dist):
        coord = self.position()
        width = self.canvas.winfo_width()
        if coord[2] + dist <= width and coord[0] + dist >= 0:
            super(Paddle, self).move(dist, 0)
            if self.ball is not None:
                self.ball.move(dist, 0)

class Brick(PlayComponent):
    colorArray = {1: 'lightsteelblue', 2: 'royalblue', 3: 'blue'}

    def __init__(self, canvas, x, y, hits):
        self.width = 60
        self.height = 20
        self.hits = hits
        color = Brick.colorArray[hits]
        item = canvas.create_rectangle(x - self.width/2, y - self.height/2,
                                       x + self.width/2, y + self.height/2,
                                       fill=color, tag='brick')
        super(Brick, self).__init__(canvas, item)

    def hit(self):
        self.hits -= 1
        if self.hits == 0:
            self.delete()
        else:
            self.canvas.itemconfig(self.item, fill=Brick.colorArray[self.hits])

class Ball(PlayComponent):
    def __init__(self, canvas, x, y):
        self.radius = 6
        self.speed = 8
        self.direction = [-1, 1]
        item = canvas.create_oval(x - self.radius, y - self.radius,
                                  x + self.radius, y + self.radius, fill='red')
        super(Ball, self).__init__(canvas, item)

    def update(self):
        coord = self.position()
        width = self.canvas.winfo_width()
        if coord[1] <= 0:
            self.direction[1] *= -1
        if coord[2] >= width or coord[0] <= 0:
            self.direction[0] *= -1

        x = self.direction[0] * self.speed
        y = self.direction[1] * self.speed
        self.move(x, y)

    def intersect(self, components):
        coord = self.position()
        x = (coord[0] + coord[2]) * 0.5

        if len(components) == 1:
            component = components[0]
            coord = component.position()
            if x < coord[0]:
                self.direction[0] = -1
            elif x > coord[2]:
                self.direction[0] = 1
            else:
                self.direction[1] *= -1
        elif len(components) > 1:
            self.direction[1] *= -1

        for component in components:
            if isinstance(component, Brick):
                component.hit()

class Game(tk.Frame):
    def __init__(self, master):
        super(Game, self).__init__(master)
        self.lives = 3
        self.width = 1000
        self.height = 400

        self.canvas = tk.Canvas(self, bg='cornsilk', width=self.width, height=self.height)
        self.canvas.pack()
        self.pack()

        self.items = {}
        self.ball = None
        self.paddle = Paddle(self.canvas, self.width/2, 320)
        self.items[self.paddle.item] = self.paddle

        for x in range(100, self.width - 100, 60):
            self.display_brick(x+20, 50, 2)
            self.display_brick(x+20, 70, 1)
            self.display_brick(x+20, 120, 1)

        self.hud = None
        self.init_game()
        self.canvas.focus_set()
        self.canvas.bind('<Left>', lambda _: self.paddle.move(-30))
        self.canvas.bind('<Right>', lambda _: self.paddle.move(30))

    def init_game(self):
        self.update_lives_text()
        self.display_ball()
        self.text = self.draw_text(self.width/2, self.height/2, 'Press "S" for start')
        self.canvas.bind('<s>', lambda _: self.start_game())

    def display_ball(self):
        if self.ball is not None:
            self.ball.delete()
        paddle_coords = self.paddle.position()
        x = (paddle_coords[0] + paddle_coords[2]) * 0.5
        self.ball = Ball(self.canvas, x, 310)
        self.paddle.set_ball(self.ball)

    def display_brick(self, x, y, hits):
        brick = Brick(self.canvas, x, y, hits)
        self.items[brick.item] = brick

    def draw_text(self, x, y, text, size='50'):
        font = ('Arial', size)
        return self.canvas.create_text(x, y, text=text, font=font)

    def update_lives_text(self):
        text = 'Lives: %s' % self.lives
        if self.hud is None:
            self.hud = self.draw_text(50, 20, text, 15)
        else:
            self.canvas.itemconfig(self.hud, text=text)

    def start_game(self):
        self.canvas.unbind('<s>')
        self.canvas.delete(self.text)
        self.paddle.ball = None
        self.game_loop()

    def game_loop(self):
        self.verify_inter()
        num_bricks = len(self.canvas.find_withtag('brick'))

        if num_bricks == 0:
            self.ball.speed = None
            self.draw_text(self.width/2, self.height/2, "You Win!")
        elif self.ball.position()[3] >= self.height:
            self.ball.speed = None
            self.lives -= 1
            if self.lives == 0:
                self.draw_text(self.width/2, self.height/2, "You Lost. Game Over!")
            else:
                self.after(1000, self.init_game())
        else:
            self.ball.update()
            self.after(50, self.game_loop)

    def verify_inter(self):
        ball_coords = self.ball.position()
        items = self.canvas.find_overlapping(*ball_coords)
        objects = [self.items[x] for x in items if x in self.items]
        self.ball.intersect(objects)
        
if __name__ =='__main__':
    root = tk.Tk()
    root.title('Brick Breaker')
    game = Game(root)
    game.mainloop()

## Tkinter - Catchfall 接蛋遊戲

In [2]:
from itertools import cycle
from random import randrange
from tkinter import Tk, Canvas, messagebox, font

# Set the canvas dimensions
canvas_width = 800
canvas_height = 400

# Initialize the main window
window = Tk()
canvas = Canvas(window, width=canvas_width, height=canvas_height, background='deep sky blue')
canvas.create_rectangle(-5, canvas_height - 100, canvas_width + 5, canvas_height + 5, fill='sea green', width=0)
canvas.create_oval(-80, -80, 120, 120, fill='orange', width=0)
canvas.pack()

# Define various game parameters
color_cycle = cycle(['light blue', 'light pink', 'light yellow', 'light green', 'red', 'blue', 'green', 'black'])
egg_width = 45
egg_height = 55
egg_score = 10
egg_speed = 500
egg_interval = 4000
difficulty_factor = 0.95

catcher_color = 'blue'
catcher_width = 100
catcher_height = 100
catcher_start_x = canvas_width / 2 - catcher_width / 2
catcher_start_y = canvas_height - catcher_height - 20
catcher_start_x2 = catcher_start_x + catcher_width
catcher_start_y2 = catcher_start_y + catcher_height

# Create the catcher
catcher = canvas.create_arc(catcher_start_x, catcher_start_y, catcher_start_x2, catcher_start_y2, start=200, extent=140,
                            style='arc', outline=catcher_color, width=3)

# Initialize game score and lives remaining
score = 0
score_text = canvas.create_text(10, 10, anchor='nw', font=('Arial', 18, 'bold'), fill='darkblue', text='Score: ' + str(score))

lives_remaining = 3
lives_text = canvas.create_text(canvas_width - 10, 10, anchor='ne', font=('Arial', 18, 'bold'), fill='darkblue',
                                text='Lives: ' + str(lives_remaining))

eggs = []

# Function to create new eggs
def create_eggs():
    x = randrange(10, 740)
    y = 40
    new_egg = canvas.create_oval(x, y, x + egg_width, y + egg_height, fill=next(color_cycle), width=0)
    eggs.append(new_egg)
    window.after(egg_interval, create_eggs)

# Function to move eggs
def move_eggs():
    for egg in eggs:
        (egg_x, egg_y, egg_x2, egg_y2) = canvas.coords(egg)
        canvas.move(egg, 0, 10)
        if egg_y2 > canvas_height:
            egg_dropped(egg)
    window.after(egg_speed, move_eggs)

# Function to handle dropped eggs
def egg_dropped(egg):
    eggs.remove(egg)
    canvas.delete(egg)
    lose_a_life()
    if lives_remaining == 0:
        messagebox.showinfo('GAME OVER!', 'Final Score: ' + str(score))
        window.destroy()

# Function to decrease the number of lives
def lose_a_life():
    global lives_remaining
    lives_remaining -= 1
    canvas.itemconfigure(lives_text, text='Lives: ' + str(lives_remaining))

# Function to check if the catcher catches an egg
def catch_check():
    (catcher_x, catcher_y, catcher_x2, catcher_y2) = canvas.coords(catcher)
    for egg in eggs:
        (egg_x, egg_y, egg_x2, egg_y2) = canvas.coords(egg)
        if catcher_x < egg_x and egg_x2 < catcher_x2 and catcher_y2 - egg_y2 < 40:
            eggs.remove(egg)
            canvas.delete(egg)
            increase_score(egg_score)
    window.after(100, catch_check)

# Function to increase the score and adjust game parameters
def increase_score(points):
    global score, egg_speed, egg_interval
    score += points
    egg_speed = int(egg_speed * difficulty_factor)
    egg_interval = int(egg_interval * difficulty_factor)
    canvas.itemconfigure(score_text, text='Score: ' + str(score))

# Functions to move the catcher left and right
def move_left(event):
    (x1, y1, x2, y2) = canvas.coords(catcher)
    if x1 > 0:
        canvas.move(catcher, -20, 0)

def move_right(event):
    (x1, y1, x2, y2) = canvas.coords(catcher)
    if x2 < canvas_width:
        canvas.move(catcher, 20, 0)

# Bind left and right arrow keys to move functions
canvas.bind('<Left>', move_left)
canvas.bind('<Right>', move_right)
canvas.focus_set()

# Start the game loop
window.after(1000, create_eggs)
window.after(1000, move_eggs)
window.after(1000, catch_check)

# Start the main event loop
window.mainloop()


## Tkinter - Match Card 翻翻牌

In [3]:
import random
import time
from tkinter import Tk, Button, DISABLED

# Function to reveal symbols on the buttons
def show_symbol(x, y):
    global first
    global previous_x, previous_y
    buttons[x, y]['text'] = button_symbols[x, y]
    buttons[x, y].update_idletasks()

    if first:
        previous_x = x
        previous_y = y
        first = False
    elif previous_x != x or previous_y != y:
        if buttons[previous_x, previous_y]['text'] != buttons[x, y]['text']:
            time.sleep(0.5)
            buttons[previous_x, previous_y]['text'] = ' '
            buttons[x, y]['text'] = ' '
        else:
            buttons[previous_x, previous_y]['command'] = DISABLED
            buttons[x, y]['command'] = DISABLED
        first = True

# Create the main window
win = Tk()
win.title('Matchmaker')
win.resizable(width=False, height=False)
first = True
previous_x = 0
previous_y = 0
buttons = {}
button_symbols = {}
symbols = [u'\u2702', u'\u2705', u'\u2708', u'\u2709', u'\u270A', u'\u270B',
           u'\u270C', u'\u270F', u'\u2712', u'\u2714', u'\u2716', u'\u2728',
           u'\u2702', u'\u2705', u'\u2708', u'\u2709', u'\u270A', u'\u270B',
           u'\u270C', u'\u270F', u'\u2712', u'\u2714', u'\u2716', u'\u2728']

# Shuffle the symbols
random.shuffle(symbols)

# Create buttons and associate them with symbols
for x in range(6):
    for y in range(4):
        button = Button(command=lambda x=x, y=y: show_symbol(x, y), width=10, height=8)
        button.grid(column=x, row=y)
        buttons[x, y] = button
        button_symbols[x, y] = symbols.pop()

# Start the main event loop
win.mainloop()

## Screen Pet

In [4]:
from tkinter import Tk, HIDDEN, NORMAL, Canvas

def toggle_eyes():
    current_color = canvas.itemcget(eye_left, 'fill')
    new_color = canvas.body_color if current_color == 'white' else 'white'
    current_state = canvas.itemcget(pupil_left, 'state')
    new_state = NORMAL if current_state == HIDDEN else HIDDEN
    canvas.itemconfigure(pupil_left, state=new_state)
    canvas.itemconfigure(pupil_right, state=new_state)
    canvas.itemconfigure(eye_left, fill=new_color)
    canvas.itemconfigure(eye_right, fill=new_color)

def blink():
    toggle_eyes()
    window.after(250, toggle_eyes)
    window.after(3000, blink)

def toggle_pupils():
    if not canvas.crossed_eyes:
        canvas.move(pupil_left, 10, -5)
        canvas.move(pupil_right, -10, -5)
        canvas.crossed_eyes = True
    else:
        canvas.move(pupil_left, -10, 5)
        canvas.move(pupil_right, 10, 5)
        canvas.crossed_eyes = False

def toggle_tongue():
    if not canvas.tongue_out:
        canvas.itemconfigure(tongue_tip, state=NORMAL)
        canvas.itemconfigure(tongue_main, state=NORMAL)
        canvas.tongue_out = True
    else:
        canvas.itemconfigure(tongue_tip, state=HIDDEN)
        canvas.itemconfigure(tongue_main, state=HIDDEN)
        canvas.tongue_out = False

def cheeky(event):
    toggle_tongue()
    toggle_pupils()
    hide_happy(event)
    window.after(1000, toggle_tongue)
    window.after(1000, toggle_pupils)
    return

def show_happy(event):
    if (20 <= event.x <= 350) and (20 <= event.y <= 350):
        canvas.itemconfigure(cheek_left, state=NORMAL)
        canvas.itemconfigure(cheek_right, state=NORMAL)
        canvas.itemconfigure(mouth_happy, state=NORMAL)
        canvas.itemconfigure(mouth_normal, state=HIDDEN)
        canvas.itemconfigure(mouth_sad, state=HIDDEN)
        canvas.happy_level = 10
    return

def hide_happy(event):
    canvas.itemconfigure(cheek_left, state=HIDDEN)
    canvas.itemconfigure(cheek_right, state=HIDDEN)
    canvas.itemconfigure(mouth_happy, state=HIDDEN)
    canvas.itemconfigure(mouth_normal, state=NORMAL)
    canvas.itemconfigure(mouth_sad, state=HIDDEN)
    return

def sad():
    if canvas.happy_level == 0:
        canvas.itemconfigure(mouth_happy, state=HIDDEN)
        canvas.itemconfigure(mouth_normal, state=HIDDEN)
        canvas.itemconfigure(mouth_sad, state=NORMAL)
    else:
        canvas.happy_level -= 1
    window.after(500, sad)

window = Tk()

canvas = Canvas(window, width=400, height=400)
canvas.configure(bg='dark blue', highlightthickness=0)

canvas.body_color = 'SkyBlue1'

body = canvas.create_oval(35, 20, 365, 350, outline=canvas.body_color, fill=canvas.body_color)
foot_left = canvas.create_oval(65, 320, 145, 360, outline=canvas.body_color, fill=canvas.body_color)
foot_right = canvas.create_oval(250, 320, 330, 360, outline=canvas.body_color, fill=canvas.body_color)

ear_left = canvas.create_polygon(75, 80, 75, 10, 165, 70, outline=canvas.body_color, fill=canvas.body_color)
ear_right = canvas.create_polygon(255, 45, 325, 10, 320, 70, outline=canvas.body_color, fill=canvas.body_color)

eye_left = canvas.create_oval(130, 110, 160, 170, outline='black', fill='white')
pupil_left = canvas.create_oval(140, 145, 150, 155, outline='black', fill='black')
eye_right = canvas.create_oval(230, 110, 260, 170, outline='black', fill='white')
pupil_right = canvas.create_oval(240, 145, 250, 155, outline='black', fill='black')

mouth_normal = canvas.create_line(170, 250, 200, 272, 230, 250, smooth=1, width=2, state=NORMAL)
mouth_happy = canvas.create_line(170, 250, 200, 282, 230, 250, smooth=1, width=2, state=HIDDEN)
mouth_sad = canvas.create_line(170, 250, 200, 232, 230, 250, smooth=1, width=2, state=HIDDEN)

tongue_main = canvas.create_rectangle(170, 250, 230, 290, outline='red', fill='red', state=HIDDEN)
tongue_tip = canvas.create_oval(170, 285, 230, 300, outline='red', fill='red', state=HIDDEN)

cheek_left = canvas.create_oval(70, 180, 120, 230, outline='pink', fill='pink', state=HIDDEN)
cheek_right = canvas.create_oval(280, 180, 330, 230, outline='pink', fill='pink', state=HIDDEN)

canvas.pack()

canvas.bind('<Motion>', show_happy)
canvas.bind('<Leave>', hide_happy)
canvas.bind('<Double-1>', cheeky)

canvas.crossed_eyes = False
canvas.tongue_out = False
canvas.happy_level = 10

window.after(1000, blink)
window.after(5000, sad)
window.mainloop()

## Tkinter - Easy Caculater

In [5]:
from tkinter import *
from functools import partial

def button_pressed(value):
    # Append the clicked button value to the expression field
    expression_field_value.set(expression_field_value.get() + str(value))

def equal_pressed():
    try:
        # Evaluate the expression and set the result in the expression field
        result = eval(expression_field_value.get())
        expression_field_value.set(result)
    except ZeroDivisionError:
        # Handle division by zero error
        expression_field_value.set("Division by zero error")

def clear_pressed():
    # Clear the expression field
    expression_field_value.set("")

if __name__ == "__main__":
    window = Tk()
    window.title("My Little Calculator")

    expression_field_value = StringVar()
    expression_field = Entry(window, width=30, textvariable=expression_field_value)
    expression_field.grid(row=0, column=0, columnspan=4)

    button_rows = [["1", "2", "3", "*"],
                   ["4", "5", "6", "-"],
                   ["7", "8", "9", "+"],
                   ["0", "/"]]

    for row, buttons in enumerate(button_rows):
        for col, button_value in enumerate(buttons):
            when_pressed = partial(button_pressed, button_value)
            button = Button(window, text=button_value, height=3, width=3, borderwidth=1, command=when_pressed)
            button.grid(row=row + 1, column=col if button_value != "/" else 3, sticky="ew")

    equal_button = Button(window, text="=", height=3, width=3, borderwidth=1, command=equal_pressed)
    equal_button.grid(row=4, column=1, sticky="ew")

    clear_button = Button(window, text="C", height=3, width=3, borderwidth=1, command=clear_pressed)
    clear_button.grid(row=4, column=2, sticky="ew")

    window.mainloop()

## Tkinter - Advance Caculater 2

In [6]:
from tkinter import *

class CalculatorApp(Frame):
    def __init__(self, master):
        """ Initialize the Calculator Frame. """
        super(CalculatorApp, self).__init__(master)
        self.equation = ""
        self.user_input = StringVar()
        self.grid()
        self.create_widgets()

    def create_widgets(self):
        """ Create all the buttons for the calculator. """
        # User input displayed as an Entry widget.

        self.input_display = Entry(self, bg="#5BC8AC", bd=29,
                                   insertwidth=4, width=24,
                                   font=("Verdana", 20, "bold"), textvariable=self.user_input, justify=RIGHT)
        self.input_display.grid(columnspan=4)
        self.input_display.insert(0, "0")

        # Buttons for digits
        digits = [
            "7", "8", "9",
            "4", "5", "6",
            "1", "2", "3",
            "0"
        ]

        for i, digit in enumerate(digits):
            row = i // 3 + 2
            col = i % 3
            button = Button(self, bg="#98DBC6", bd=12, text=digit, padx=33, pady=25,
                            font=("Helvetica", 20, "bold"), command=lambda d=digit: self.button_click(d))
            button.grid(row=row, column=col, sticky=W)

        # Operator buttons
        operators = [
            "+", "-", "*", "/"
        ]

        for i, operator in enumerate(operators):
            button = Button(self, bg="#98DBC6", bd=12, text=operator, padx=36, pady=25,
                            font=("Helvetica", 20, "bold"), command=lambda o=operator: self.button_click(o))
            button.grid(row=i + 2, column=3, sticky=W)

        # Equal button
        equal_button = Button(self, bg="#E6D72A", bd=12, text="=", padx=100, pady=25,
                              font=("Helvetica", 20, "bold"), command=self.calculate_task)
        equal_button.grid(row=5, column=1, sticky=W, columnspan=2)

        # Clear Button
        clear_button = Button(self, bg="#E6D72A", bd=12, text="AC", font=("Helvetica", 20, "bold"),
                              width=28, padx=7, command=self.clear_display)
        clear_button.grid(row=1, columnspan=4, sticky=W)

    def button_click(self, value):
        self.equation = str(self.equation) + str(value)
        self.user_input.set(self.equation)

    def calculate_task(self):
        data = self.input_display.get()
        try:
            answer = eval(data)
            self.display_result(answer)
            self.equation = str(answer)

        except SyntaxError as e:
            self.display_result("Invalid Syntax!")
            self.equation = ""

    def display_result(self, value):
        self.input_display.delete(0, END)
        self.input_display.insert(0, value)

    def clear_display(self):
        self.equation = ""
        self.input_display.delete(0, END)
        self.input_display.insert(0, "0")


calculator = Tk()
calculator.title("Calculator")
app = CalculatorApp(calculator)
# Make window fixed (cannot be resized)
calculator.resizable(width=False, height=False)
calculator.mainloop()


## Digital Clock

In [7]:
from time import strftime
from tkinter import Label, Tk

# Window configuration for the clock
window = Tk()
window.title("Digital Clock")
window.geometry("200x80")
window.configure(bg="red")
window.resizable(False, False)

# Label configuration for displaying the time
clock_label = Label(window, bg="red", fg="white", font=("Times", 30, 'bold'), relief='flat')
clock_label.place(x=20, y=20)

# Function to update the label with the current time
def update_label():
    current_time = strftime('%H:%M:%S')
    clock_label.configure(text=current_time)
    clock_label.after(1000, update_label)  # Update every 1 second (1000 milliseconds)

update_label()
window.mainloop()
